In [ ]:
import pandas as pd
import numpy as np
from pytz import utc
from feast import Client, FeatureSet, Entity, ValueType
from google.protobuf.duration_pb2 import Duration
from datetime import datetime, timedelta
from random import randrange, randint
import os

In [ ]:
project_name = 'product_matching_team'
feature_set_name = 'product_features'

FEATURE_NAMES = ['texture_embedding', 'spoc_embedding', 'local_histogram', 'ccv']
FEATURE_TYPES = [ValueType.DOUBLE_LIST, ValueType.DOUBLE_LIST, ValueType.DOUBLE_LIST, ValueType.DOUBLE_LIST]

In [ ]:
CORE_URL = os.getenv('FEAST_CORE_URL')
BATCH_SERVING_URL = os.getenv('FEAST_BATCH_SERVING_URL')
ONLINE_SERVING_URL = os.getenv('FEAST_ONLINE_SERVING_URL')
print(CORE_URL, BATCH_SERVING_URL, ONLINE_SERVING_URL)

In [ ]:
def construct_project(project_name):
    client = Client(core_url=CORE_URL, serving_url=BATCH_SERVING_URL, project=project_name)
    if project_name not in client.list_projects():
        print('constructing project: {}'.format(project_name))
        client.create_project(project_name)
    else:
        print('project already exists: {}'.format(project_name))
    return client

In [ ]:
def get_product_features(client, project_name, products, feature_refs):

    entity_rows = pd.DataFrame(
        {
            "datetime": [datetime.utcnow().replace(tzinfo=utc) for _ in range(len(products))],
            "product_id": [product for product in products],
        }
    )
    
    job = client.get_batch_features(
        feature_refs=feature_refs,
        entity_rows=entity_rows,
        default_project=project_name,
    )
    df = job.to_dataframe()
    return df

In [ ]:
products = [13485106, 14939385, 14550846, 14272276]

client = construct_project(project_name=project_name)

In [ ]:
df = get_product_features(client, project_name, products, FEATURE_NAMES)

In [ ]:
df